## Uso de Clusters junto con Aprendizaje Supervisado - Evaluando hiperparametros

En esta ocasion los siguientes algoritmos de aprendizaje supervisado
. Regresion Logistica
. Pipeline con clusterin(k-means) y la regresion logistica
. Modelacion Hyperparametros GridSearchCV

En la cual evaluaremos la precisión de los modelos y encontrar los mejores parametros para obtener un mejor rendimiento luego para nustra modelación 

La agrupación puede ser un enfoque para la reduccion de la dimensionalidad, en particular comoo un paso de preprocesamiento antes de un algoritmo de aprendizaje supervisado. Por ejemplo abordaremos el conjunto de datos de digitos, que es un conjunto de datos simple de similar al NINIST que contiene 1797 imagenes en escala de grises de 8x8 que representan los digitos del 0 al 9

In [1]:
from sklearn.datasets import load_digits

In [2]:
X_digits, y_digits = load_digits(return_X_y=True)

Hacemos la particion del conjunto de datos para entrenar y probar nuestros modelos

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train,y_test = train_test_split(X_digits,y_digits,test_size=0.3,random_state=42)

Usaremos el Modelo de Regresion Logistica para ajustar nuestro modelo y evaluarlo con el conjunto de datos de prueba

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
#Iniciamos nuestro modelos con algunos parametros
log_reg=LogisticRegression(multi_class="ovr",solver="lbfgs",max_iter=5000,random_state=42)

#Entrenamos nuestro modelo
log_reg.fit(X_train,y_train)

LogisticRegression(max_iter=5000, multi_class='ovr', random_state=42)

Evaluamos la precision de nuestro modelo con el conjunto de prueba

In [9]:
log_reg_score=log_reg.score(X_test,y_test)

In [10]:
log_reg_score

0.9592592592592593

De acuerdo con nuestro resultado, podemos observar que la precision(accuracy) de nuestro modelo es de 95.92%. Veamos si podemos mejorar la precision utilizando Clustering(K-Means) como paso previo al procesamiento. Crearemos un Pipeline que primero agrupará el conjunto de entrenamiento en 50 grupos y reemplazaremos las imagenes con sus distancia a estos 50 grupos, luego aplicaremos el modelo de regresión logistica

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

In [12]:
pipeline=Pipeline([
    
    ("kmeans",KMeans(n_clusters=50, random_state=42)),
    ("log_reg",LogisticRegression(multi_class="ovr",solver="lbfgs",max_iter=5000, random_state=42)),   
])

#Entrenamiento del pipeline
pipeline.fit(X_train,y_train)

Pipeline(steps=[('kmeans', KMeans(n_clusters=50, random_state=42)),
                ('log_reg',
                 LogisticRegression(max_iter=5000, multi_class='ovr',
                                    random_state=42))])

Evaluamos la precision de nuestro modelo con el conjunto de Prueba

In [13]:
pipeline_score=pipeline.score(X_test,y_test)
pipeline_score

0.975925925925926

Cuanto bajo la tasa de error ?

In [14]:
1 - (1 - pipeline_score)/(1 - log_reg_score)

0.4090909090909103

Que pasó? Pudimos reducir la tasa de error en un 40.9%

Pero podemos elegir el numero de Cluster K completamente arbitraria, seguramente podemos hacerlo mejor. Dado que K-means es un solo paso de procesamiento de una clasificacion de Pipeline encontrar un buen valor de k es mucho mas simple que antes: en esta ocasión no es necesario realizar un analisis de silueta  o minimizar la inercia, el mejor valor de k es simplemente el que da como resultado el mejor rendimiento de clasificacion durante la validacion cruzada. Usaremos GridSearchCV para encontrar la cantidad óptima de clusters

In [15]:
from sklearn.model_selection import GridSearchCV

**Atención:** El siguiente codigo puede durar mas de 20 minutos en correr, dependiendo de nuestra compu

In [17]:
param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline,param_grid,cv=3,verbose=2)
grid_clf.fit(X_train,y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] kmeans__n_clusters=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................. kmeans__n_clusters=2, total=   0.4s
[CV] kmeans__n_clusters=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ............................. kmeans__n_clusters=2, total=   0.3s
[CV] kmeans__n_clusters=2 ............................................
[CV] ............................. kmeans__n_clusters=2, total=   0.3s
[CV] kmeans__n_clusters=3 ............................................
[CV] ............................. kmeans__n_clusters=3, total=   0.4s
[CV] kmeans__n_clusters=3 ............................................
[CV] ............................. kmeans__n_clusters=3, total=   0.4s
[CV] kmeans__n_clusters=3 ............................................
[CV] ............................. kmeans__n_clusters=3, total=   0.4s
[CV] kmeans__n_clusters=4 ............................................
[CV] ............................. kmeans__n_clusters=4, total=   0.4s
[CV] kmeans__n_clusters=4 ............................................
[CV] ............................. kmeans__n_clusters=4, total=   0.5s
[CV] kmeans__n_clusters=4 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done 294 out of 294 | elapsed: 36.1min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans',
                                        KMeans(n_clusters=50, random_state=42)),
                                       ('log_reg',
                                        LogisticRegression(max_iter=5000,
                                                           multi_class='ovr',
                                                           random_state=42))]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

Veamos cual es el mejor numero de cluster y el desempeño del resultado de Pipeline

In [18]:
grid_clf.best_params_

{'kmeans__n_clusters': 73}

In [19]:
grid_clf.score(X_test,y_test)

0.9777777777777777

Con un k=73 clusters, obtenemos un aumento en la precision, alcanzando un 97.7% de precision en el conjunto de prueba